In [1]:
import torch
import math
torch.set_grad_enabled(False)

In [2]:
def generate_set():
    training_set = torch.empty(1000,2).uniform_(0,1)  #x et y
    training_classes = torch.empty(1000)
    testing_set = torch.empty(1000,2).uniform_(0,1)  #x et y
    testing_classes = torch.empty(1000)

    r = torch.empty(1,1).fill_(1/(2*math.pi)).pow(1/2)

    for i in range (1000):
        if ((training_set[i] - torch.Tensor([0.5,0.5])).pow(2).sum()).pow(1/2).item() < r.item():
            training_classes[i] = 1
        else:
            training_classes[i] = 0

        if ((testing_set[i] - torch.Tensor([0.5,0.5])).pow(2).sum()).pow(1/2).item() < r.item():
            testing_classes[i] = 1
        else:
            testing_classes[i] = 0
    return training_set, training_classes, testing_set, testing_classes

In [3]:
training_set, training_classes, testing_set, testing_classes = generate_set()

In [4]:
import matplotlib.pyplot as plt
plt.figure('1')
x = training_set[:,0]
y = training_set[:,1]
plt.scatter(x[training_classes==1], y[training_classes==1], color='r')
plt.scatter(x[training_classes==0], y[training_classes==0], color='b')

print(sum(training_classes==0))
print(sum(training_classes==1))
plt.show()

tensor(516)
tensor(484)


<Figure size 640x480 with 1 Axes>

# LINEAR

In [5]:
#LINEAR MODULE (FULLY CONNECTED LAYER)
#mini batch : x[mini_batch_size * D]D=2
class Linear(object):
    def __init__(self, dimension, nb_data_in, nb_data_out):
        #x = nb_data_in x dim in my calculation, but here transpose (dim x nb_data_in)
        #y = nb_data_out x dim in my calculation, but here transpose (dim x nb_data_out)
        k = math.sqrt(1/nb_data_in)
        self.weight = torch.empty(nb_data_out,nb_data_in).uniform_(-k,k)
        self.bias = torch.empty(nb_data_out,dimension).uniform_(-k,k)
        self.grad_weight = None
        self.grad_bias = None
        self.input = None
        
    def updateparam(self, lr):
        for i in range(len(self.grad_weight)): 
            self.weight -= lr * self.grad_weight[i]
            self.bias -= lr * self.grad_bias[i]
    
    def forward(self , *input, nb_layer):
        #All the calculation were done considering x = nb_data_in x dim (as seen in the lesson)
        #So to simplify the comprehension, we do a first transpose, do the the calculations with this
        #then transpose again after, to have the correct dimension output

        #save x with dim = dim x nb_data_in
        #1er étage:
        #forward(x) ->(x,)
        #2ème ou plus:
        #(x,) -> ((x,),)
        if(nb_layer > 1):
            input = input[0]
            #pour le premier, on aura en entrée un ou plusieurs tensors (donne tuple quand rentré dans f)
        
        #save x with dim = dim x nb_data_in    
        self.input = input
        output = []
        
        for x in input:
            #x = dim x nb_data_in -> nb_data_in x dim
            x_correct_dim = x.t()
            #y = nb_data_out x dim
            y_correct_dim = None

            #dim analysis: [nb_data_out x nb_data_in] x [nb_data_in x dim] + [nb_data_out x dim]
            y_correct_dim = ((self.weight).matmul(x_correct_dim)+self.bias)

            #append y = [dim x nb_data_out] to respect lin module
            output.append(y_correct_dim.t()) 
        return tuple(output)
    
    def backward(self, *gradwrtoutput):
        gradwrtoutput = gradwrtoutput[0]
        dl_dx  = None
        dl_dw = []
        dl_db = []
        gradaccumulated = []
        
        for i in range (len(gradwrtoutput)):
            #X = dim x nb_data_in -> nb_data_in x dim
            x_correct_dim = self.input[i].t()
            
            #X = dim x nb_data_out -> nb_data_out x dim
            grad_correct_dim = gradwrtoutput[i].t()
            
            #dl_dx = w^T x dl_dy (gradwrtoutput)
            #dim analysis: nb_data_in x dim = [nb_data_in x nb_data_out] x [nb_data_out x dim]
            dl_dx = (self.weight).t().matmul(grad_correct_dim)
            #nb_data_in x dim -> dim x nb_data_in
            gradaccumulated.append(dl_dx.t())
            
            #dl_db = dl_dy
            #dim analysis: nb_data_out x dim (car b = nb_data_out x dim)
            dl_db.append(grad_correct_dim)
            
            #dl_dw = dl_dy x X^T
            #dim analysis: nb_data_out x nb_data_in = [nb_data_out x dim] x [dim x nb_data_in]
            dl_dw.append(grad_correct_dim.matmul(x_correct_dim.t()))
            
        self.grad_weight = dl_dw
        self.grad_bias = dl_db

        return tuple(gradaccumulated)
        
    def param(self):
        output = [[self.weight, self.grad_weight], [self.bias, self.grad_bias]]
        return output
    
    #https://pytorch.org/docs/stable/nn.html#linear-layers

# RELU

In [6]:
#RELU MODULE
class ReLU():
    def __init__(self):
        self.input = None
    
    def forward(self, *input):
        input = input[0]
        liste = []
        self.input = input

        for x in input:
            liste.append(torch.max(x,torch.zeros_like(x))) 
        output = tuple(liste)
        return output
        
    def backward(self, *gradwrtoutput): 
        #add input as param ? 
        #No, parameters are w and b when linear, but input is saved has if we have to give backward and input
        #in the case they're both tuple of tensors of size not know
        #it's complicated (maybe impossible) to code it with the * (let the size be whatever we want)
        gradwrtoutput = gradwrtoutput[0]
        derivative = []
        gradaccumulated = []

        for x in self.input:
            dx = (x>=0).float()
            derivative.append(dx)
        for i in range (len(derivative)):
            gradaccumulated.append(derivative[i]*gradwrtoutput[i])
        output = tuple(gradaccumulated)
        return output   

    def param(self): 
        return []
    
#backward should get as input a tensor or a tuple of tensors containing the gradient of the 
#loss with respect to the module’s output, accumulate the gradient wrt the parameters, 
#and return a tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.

# TANH

In [7]:
#TANH MODULE
class Tanh():
    def __init__(self):
        self.input = None
        
    def forward(self, *input):
        input = input[0]
        output = []
        self.input = input
        for x in input:
            output.append(torch.tanh(x)) 
        return tuple(output)
    
    def backward(self, *gradwrtoutput):
        gradwrtoutput = gradwrtoutput[0]
        output = []
        for i in range (len(gradwrtoutput)):
            # derivative = (1 - torch.tanh(x).pow(2))
            output.append((1 - torch.tanh(self.input[i]).pow(2))*gradwrtoutput[i])
        return tuple(output)   

    def param(self):
        return [] #Pas de param

# LOSSMSE

In [8]:
#LOSSMSE MODULE
    
class LossMSE():
    #def __init__(self):
        #Besoin d'aucun je pense
        #self.loss = None
        #self.gradloss = None
    
    def forward(self, *input, target): 
        #Input étant seul a été rangé dans un tuple
        #target a été mis dans un tuple AVANT (comme l'appel de la fonction ne le range pas dans un tuple)

        #1er étage:
        #forward(x,target) ->(x,), target= (t,) (tuple en dehors)
        #t = [0 1] => 0
        
        liste = torch.Tensor([])
        #Utilisation de cette écriture pour pouvoir faire somme sur "liste" (l = [] marche pas avec sum())
        for i in range (len(input)):
            liste = torch.cat((liste,(input[i]-target[i]).pow(2)))
        loss = torch.sum(liste)/len(input)
        #self.loss = loss même pas besoin
        return loss
        
    def backward(self, *input, target):
        gradaccumulated = []
        dloss = []
        
        for i,x in enumerate(input):
            dloss.append(2*(x - target[i])/len(input))
        return tuple(dloss)

    def param(self):
        return [] #No param

In [9]:
def class_into_2(classes):
    t = torch.empty(classes.size(0),2).zero_()
    for n in range (classes.size(0)):
        t[n,int(classes[n].item())] = 1
    return t

In [10]:
x = torch.empty(1,2).fill_(0)
y = torch.empty(2,1).fill_(0)
x[0][1]=1
y[1][0]=1
print(x)
print(x.max(0))
#print(x.max(0)[1])
#torch.max(x,0)
#print(x.max(0,0))
print()
print(y)
print(y.max(0))
print(y.max(0)[1].item())
#print(x.max(0)[1].item())

tensor([[0., 1.]])
torch.return_types.max(
values=tensor([0., 1.]),
indices=tensor([0, 0]))

tensor([[0.],
        [1.]])
torch.return_types.max(
values=tensor([1.]),
indices=tensor([1]))
1


In [11]:
print(training_classes.size())
print(training_classes[1])
train_target = class_into_2(training_classes);
print(train_target.size())
print(train_target[1])
print()
print(train_target[1].max(0))
print(train_target[1].max(0)[1].item())

#test_target = class_into_2(testing_classes);

torch.Size([1000])
tensor(0.)
torch.Size([1000, 2])
tensor([1., 0.])

torch.return_types.max(
values=tensor(1.),
indices=tensor(0))
0


# Gradient 2 layer Tanh

In [ ]:
nb_iterations = 5
print(nb_iterations)

lin1 = Linear(1,2,25)
lin2 = Linear(1,25,2)
T1 = Tanh()
L = LossMSE()
  

for i in range (nb_iterations):
    print()
    print('iteration', i)
    loss_training = 0
    
    #1000 x 1 -> 1000 x 2:  [1 0] and [0 1]
    train_target = class_into_2(training_classes);
    test_target = class_into_2(testing_classes);
    
    #Etape Update valeur par valeur
    for j in range(training_set.size(0)):
        f1 = lin1.forward(training_set[j].unsqueeze(0), nb_layer = 1)
        t1 = T1.forward(f1)
        output = (lin2.forward(t1, nb_layer = 2))[0]
    
        loss = L.forward(output, target = tuple(train_target[j]))
        loss_training = loss_training + loss
        
        dl = L.backward(output, target = tuple(train_target[j]))
        print('dl',dl)
        bf2 = lin2.backward(dl)
        print('bf2', bf2)
        bt1 = T1.backward(bf2)
        print('bt1', bt1)
        bf1 = lin1.backward(bt1)
        print('bf1',bf1)

        lr = 0.01
        #print(lin1.param()[0][0])
        lin1.updateparam(lr)
        lin2.updateparam(lr)
        #print(lin1.param()[0][0])
    print('loss :', loss_training)

5

iteration 0
dl (tensor([[-1.8287, -2.6409]]),)
bf2 (tensor([[-0.6964, -0.6245, -0.5580, -0.5701, -0.2771,  0.2174, -0.4425, -0.2822,
          0.1635, -0.7740, -0.2276, -0.3013,  0.4160,  0.4408, -0.7695,  0.4177,
          0.2759,  0.2139,  0.3677, -0.1308,  0.4778,  0.3334, -0.0367,  0.2678,
          0.5667]]),)
bt1 (tensor([[-0.3962, -0.2106, -0.4782, -0.4932, -0.0660,  0.0747, -0.2976, -0.2795,
          0.1491, -0.6665, -0.0676, -0.2962,  0.2294,  0.1526, -0.4688,  0.3869,
          0.2238,  0.1492,  0.3605, -0.1011,  0.3796,  0.3149, -0.0281,  0.2595,
          0.3919]]),)
bf1 (tensor([[-0.0747, -0.4511]]),)
dl (tensor([[-1.4562, -2.3056]]),)
bf2 (tensor([[-0.6380, -0.4490, -0.5134, -0.4523, -0.1383,  0.1392, -0.4379, -0.2467,
          0.1648, -0.6238, -0.1428, -0.2553,  0.2991,  0.3129, -0.7045,  0.3767,
          0.2856,  0.1348,  0.3207, -0.0529,  0.4321,  0.2764, -0.0045,  0.2379,
          0.4197]]),)
bt1 (tensor([[-0.5094, -0.1982, -0.5059, -0.4444, -0.0425,  0.0554, -

          0.2124]]),)
bt1 (tensor([[-0.3997, -0.0831, -0.3040, -0.2018, -0.0011, -0.0344, -0.2549, -0.0774,
          0.1212, -0.2458,  0.0343, -0.0997,  0.0666,  0.0193, -0.4179,  0.2253,
          0.1645,  0.0371,  0.1231, -0.0176,  0.0928,  0.1070,  0.0628,  0.0500,
          0.1815]]),)
bf1 (tensor([[-0.6457, -0.5304]]),)
dl (tensor([[-0.9566, -0.9030]]),)
bf2 (tensor([[-0.3947, -0.1366, -0.2414, -0.1659, -0.0161, -0.1123, -0.2194, -0.0422,
          0.0992, -0.2454,  0.1051, -0.0772,  0.0507,  0.0188, -0.4172,  0.2135,
          0.1240,  0.0257,  0.0880, -0.0314,  0.3174,  0.0688,  0.1087,  0.0110,
          0.1845]]),)
bt1 (tensor([[-0.2359, -0.0562, -0.2144, -0.1241, -0.0057, -0.0475, -0.1658, -0.0398,
          0.0890, -0.2319,  0.0377, -0.0699,  0.0268,  0.0097, -0.2255,  0.2050,
          0.1106,  0.0223,  0.0822, -0.0293,  0.3046,  0.0679,  0.0929,  0.0109,
          0.1562]]),)
bf1 (tensor([[-0.2387, -0.5240]]),)
dl (tensor([[-0.7535, -0.9999]]),)
bf2 (tensor([[-0.3769, -0.

          0.0762]]),)
bf1 (tensor([[-0.0469, -0.1804]]),)
dl (tensor([[-0.5537, -0.1719]]),)
bf2 (tensor([[-0.1665, -0.0640, -0.0705, -0.0520, -0.0467, -0.1040, -0.0514,  0.0241,
          0.0368, -0.0853,  0.0957,  0.0010,  0.0016, -0.0200, -0.1598,  0.0797,
          0.0141,  0.0056, -0.0187, -0.0481,  0.1422, -0.0082,  0.0772, -0.0537,
          0.0920]]),)
bt1 (tensor([[-0.0737, -0.0190, -0.0512, -0.0474, -0.0085, -0.0316, -0.0274,  0.0241,
          0.0318, -0.0759,  0.0243,  0.0010,  0.0008, -0.0059, -0.0841,  0.0696,
          0.0097,  0.0032, -0.0185, -0.0326,  0.1088, -0.0077,  0.0570, -0.0520,
          0.0576]]),)
bf1 (tensor([[-0.0369, -0.1454]]),)
dl (tensor([[-0.7699, -1.4181]]),)
bf2 (tensor([[-0.4709, -0.0802, -0.3296, -0.1745,  0.1172, -0.0757, -0.3392, -0.0934,
          0.1333, -0.2864,  0.0796, -0.1253,  0.0447,  0.0243, -0.5490,  0.2685,
          0.2127,  0.0444,  0.1929,  0.0331,  0.3649,  0.1472,  0.0927,  0.1323,
          0.1668]]),)
bt1 (tensor([[-0.4594, -0.

bf1 (tensor([[0.4999, 0.6910]]),)
dl (tensor([[1.1441, 1.2090]]),)
bf2 (tensor([[ 0.4804,  0.1633,  0.2917,  0.2101,  0.0051,  0.1292,  0.2710,  0.0545,
         -0.1157,  0.3050, -0.1199,  0.1104, -0.0711, -0.0287,  0.5308, -0.2701,
         -0.1494, -0.0592, -0.1385,  0.0392, -0.4121, -0.1022, -0.1337, -0.0271,
         -0.2382]]),)
bt1 (tensor([[ 0.3347,  0.0689,  0.2746,  0.1988,  0.0015,  0.0514,  0.2122,  0.0538,
         -0.1119,  0.2646, -0.0449,  0.1104, -0.0501, -0.0107,  0.3631, -0.2466,
         -0.1344, -0.0501, -0.1382,  0.0322, -0.2312, -0.0987, -0.0989, -0.0249,
         -0.1903]]),)
bf1 (tensor([[0.4070, 0.5909]]),)
dl (tensor([[0.9253, 0.8727]]),)
bf2 (tensor([[ 0.3576,  0.1470,  0.2117,  0.1646,  0.0359,  0.0961,  0.1873,  0.0350,
         -0.0836,  0.2390, -0.0870,  0.0769, -0.0633, -0.0336,  0.3900, -0.1988,
         -0.0984, -0.0512, -0.0906,  0.0485, -0.3044, -0.0721, -0.0995, -0.0100,
         -0.1973]]),)
bt1 (tensor([[ 0.2652,  0.0687,  0.2050,  0.1443,  0.013

          0.1765]]),)
bt1 (tensor([[-0.3289, -0.0574, -0.2253, -0.1510,  0.0148, -0.0435, -0.2108, -0.0461,
          0.0881, -0.1957,  0.0423, -0.0958,  0.0441,  0.0091, -0.3469,  0.1995,
          0.1226,  0.0630,  0.1317, -0.0157,  0.1258,  0.0963,  0.0723,  0.0403,
          0.1644]]),)
bf1 (tensor([[-0.4480, -0.4708]]),)
dl (tensor([[1.3899, 1.4987]]),)
bf2 (tensor([[ 0.5951,  0.1617,  0.3429,  0.2406, -0.0325,  0.1964,  0.3312,  0.0473,
         -0.1382,  0.3507, -0.1838,  0.1373, -0.0647,  0.0015,  0.6795, -0.3424,
         -0.1737, -0.0910, -0.1835,  0.0452, -0.5582, -0.1247, -0.1927, -0.0186,
         -0.2893]]),)
bt1 (tensor([[ 0.3892,  0.0650,  0.3141,  0.2290, -0.0089,  0.0745,  0.2458,  0.0470,
         -0.1324,  0.3048, -0.0649,  0.1372, -0.0445,  0.0005,  0.4482, -0.3084,
         -0.1511, -0.0733, -0.1831,  0.0357, -0.3155, -0.1198, -0.1408, -0.0170,
         -0.2230]]),)
bf1 (tensor([[0.4478, 0.7638]]),)
dl (tensor([[1.1079, 0.8669]]),)
bf2 (tensor([[ 0.3977,  0.1517, 

...

          0.1281]]),)
bt1 (tensor([[-0.0925, -0.0099, -0.0679, -0.0660, -0.0022, -0.0403, -0.0407, -0.0003,
          0.0422, -0.0880,  0.0304, -0.0320,  0.0026,  0.0018, -0.1494,  0.1027,
          0.0188,  0.0511,  0.0448, -0.0403,  0.0795,  0.0330,  0.0702,  0.0223,
          0.0784]]),)
bf1 (tensor([[-0.0596, -0.1042]]),)
dl (tensor([[-0.7491, -0.7648]]),)
bf2 (tensor([[-0.2538, -0.0358, -0.1280, -0.0978, -0.0070, -0.1722, -0.1020, -0.0013,
          0.0705, -0.1332,  0.1551, -0.0459,  0.0015,  0.0016, -0.3835,  0.1668,
          0.0414,  0.1278,  0.0652, -0.0786,  0.1443,  0.0488,  0.1308,  0.0330,
          0.1771]]),)
bt1 (tensor([[-0.2495, -0.0261, -0.1205, -0.0861, -0.0051, -0.1135, -0.1017, -0.0011,
          0.0705, -0.1256,  0.1037, -0.0451,  0.0013,  0.0012, -0.3071,  0.1650,
          0.0398,  0.1200,  0.0651, -0.0783,  0.0910,  0.0488,  0.1133,  0.0326,
          0.1771]]),)
bf1 (tensor([[-0.2216, -0.1536]]),)
dl (tensor([[1.3840, 1.3802]]),)
bf2 (tensor([[ 0.4675,  0.05

         -0.2256]]),)
bf1 (tensor([[0.1180, 0.1827]]),)
dl (tensor([[0.9639, 0.9598]]),)
bf2 (tensor([[ 0.2844,  0.1002,  0.1471,  0.1579,  0.0700,  0.1690,  0.0999,  0.0189,
         -0.0716,  0.1819, -0.1438,  0.0767, -0.0468, -0.0493,  0.4397, -0.1988,
         -0.0351, -0.1827, -0.0946,  0.1171, -0.1614, -0.0645, -0.1422, -0.0334,
         -0.2507]]),)
bt1 (tensor([[ 0.2271,  0.0504,  0.1444,  0.1430,  0.0281,  0.0827,  0.0885,  0.0182,
         -0.0696,  0.1696, -0.0654,  0.0758, -0.0341, -0.0253,  0.3030, -0.1901,
         -0.0338, -0.1668, -0.0944,  0.1075, -0.1193, -0.0638, -0.1186, -0.0328,
         -0.2179]]),)
bf1 (tensor([[0.0351, 0.0817]]),)
dl (tensor([[-1.1576, -1.1715]]),)
bf2 (tensor([[-0.3334, -0.1359, -0.1754, -0.1978, -0.0995, -0.1862, -0.1145, -0.0281,
          0.0828, -0.2263,  0.1554, -0.0964,  0.0689,  0.0762, -0.5206,  0.2362,
          0.0390,  0.2292,  0.1184, -0.1471,  0.1838,  0.0826,  0.1617,  0.0465,
          0.3111]]),)
bt1 (tensor([[-0.3253, -0.0950, 

         -0.3291]]),)
bt1 (tensor([[ 0.3076,  0.0389,  0.1959,  0.2055,  0.0128,  0.1225,  0.1253,  0.0175,
         -0.1012,  0.2062, -0.0997,  0.1151, -0.0256, -0.0059,  0.4451, -0.2736,
         -0.0518, -0.1924, -0.1452,  0.1122, -0.1753, -0.0785, -0.1835, -0.0190,
         -0.2502]]),)
bf1 (tensor([[0.1387, 0.3032]]),)
dl (tensor([[1.0971, 1.0967]]),)
bf2 (tensor([[ 0.3279,  0.0949,  0.1614,  0.1763,  0.0608,  0.2141,  0.1141,  0.0157,
         -0.0796,  0.1942, -0.1843,  0.0923, -0.0433, -0.0354,  0.5156, -0.2340,
         -0.0372, -0.2136, -0.1163,  0.1282, -0.2147, -0.0706, -0.1787, -0.0244,
         -0.2816]]),)
bt1 (tensor([[ 0.2988,  0.0568,  0.1611,  0.1662,  0.0306,  0.1175,  0.1107,  0.0148,
         -0.0794,  0.1780, -0.1001,  0.0922, -0.0364, -0.0197,  0.3953, -0.2253,
         -0.0372, -0.2119, -0.1157,  0.1232, -0.1245, -0.0702, -0.1463, -0.0232,
         -0.2661]]),)
bf1 (tensor([[0.0734, 0.1215]]),)
dl (tensor([[-1.0450, -1.0608]]),)
bf2 (tensor([[-0.3072, -0.1047, 

          0.1987]]),)
bf1 (tensor([[-0.1023, -0.0758]]),)
dl (tensor([[1.1442, 1.1462]]),)
bf2 (tensor([[ 0.3476,  0.0917,  0.1725,  0.1762,  0.0579,  0.2304,  0.1231,  0.0143,
         -0.0877,  0.2018, -0.1997,  0.0900, -0.0367, -0.0327,  0.5460, -0.2451,
         -0.0422, -0.2203, -0.1156,  0.1332, -0.2205, -0.0732, -0.1887, -0.0310,
         -0.2910]]),)
bt1 (tensor([[ 0.2643,  0.0455,  0.1671,  0.1479,  0.0242,  0.1142,  0.1074,  0.0136,
         -0.0835,  0.1934, -0.0888,  0.0858, -0.0243, -0.0185,  0.3479, -0.2369,
         -0.0403, -0.2012, -0.1127,  0.1261, -0.1933, -0.0727, -0.1637, -0.0310,
         -0.2568]]),)
bf1 (tensor([[0.0296, 0.1564]]),)
dl (tensor([[1.0621, 1.0579]]),)
bf2 (tensor([[ 0.3101,  0.1025,  0.1553,  0.1729,  0.0730,  0.1968,  0.1049,  0.0184,
         -0.0759,  0.1917, -0.1675,  0.0881, -0.0479, -0.0459,  0.4905, -0.2224,
         -0.0337, -0.2105, -0.1098,  0.1293, -0.1956, -0.0691, -0.1662, -0.0276,
         -0.2778]]),)
bt1 (tensor([[ 0.2335,  0.0452, 

         -2.3401e-01, -8.1416e-02, -2.0389e-01, -4.2427e-02, -3.1318e-01]]),)
bt1 (tensor([[ 2.8672e-01,  4.3815e-02,  1.7202e-01,  1.3944e-01,  2.3018e-02,
          1.3047e-01,  1.1276e-01,  2.3008e-04, -9.0204e-02,  2.0427e-01,
         -1.0272e-01,  7.7741e-02, -1.8051e-02, -1.6455e-02,  3.7650e-01,
         -2.5049e-01, -3.6028e-02, -2.2906e-01, -1.1180e-01,  1.4479e-01,
         -2.0849e-01, -8.1040e-02, -1.7859e-01, -4.2392e-02, -2.8311e-01]]),)
bf1 (tensor([[0.0337, 0.1184]]),)
dl (tensor([[1.0661, 1.0645]]),)
bf2 (tensor([[ 0.3133,  0.0935,  0.1506,  0.1595,  0.0657,  0.2076,  0.1022,  0.0063,
         -0.0775,  0.1910, -0.1785,  0.0780, -0.0386, -0.0405,  0.5032, -0.2219,
         -0.0277, -0.2207, -0.1044,  0.1374, -0.1969, -0.0725, -0.1703, -0.0355,
         -0.2830]]),)
bt1 (tensor([[ 0.2829,  0.0593,  0.1502,  0.1256,  0.0409,  0.1260,  0.1007,  0.0054,
         -0.0765,  0.1850, -0.1024,  0.0722, -0.0288, -0.0294,  0.3528, -0.2199,
         -0.0277, -0.2203, -0.1022,  0.

In [ ]:
#Test après
errors_training = 0
for j in range(training_set.size(0)):
        f1 = lin1.forward(training_set[j].unsqueeze(0), nb_layer = 1)
        t1 = T1.forward(f1)
        output = (lin2.forward(t1, nb_layer = 2))[0]
        #print(output)
        pred = output.squeeze().max(0)[1].item() # 1 x 2 -> 2
        print(pred)
        if train_target[j, pred].item() < 0.5: errors_training = errors_training + 1

print(' errors training: ', errors_training)

errors_testing = 0
for j in range(testing_set.size(0)):
    f1 = lin1.forward(testing_set[j].unsqueeze(0), nb_layer = 1)
    t1 = T1.forward(f1)
    output = (lin2.forward(t1, nb_layer = 2))[0]

    #[0.3 0.7] => max = indice(1)
    pred = output.squeeze().max(0)[1].item() # 1 x 2 -> 2
    if test_target[j, pred] < 0.5: errors_testing = errors_testing + 1

print(' errors testing: ', errors_testing)

# SEQUENTIAL